## 相关资料
1. **AlphaGo背后的力量：蒙特卡洛树搜索入门指南**
  https://www.jiqizhixin.com/articles/monte-carlo-tree-search-beginners-guide

2. **如何学习蒙特卡罗树搜索（MCTS）**
  https://zhuanlan.zhihu.com/p/30458774

3. **28 天自制你的 AlphaGo (6) : 蒙特卡洛树搜索（MCTS）基础**
  https://zhuanlan.zhihu.com/p/25345778

In [1]:
import copy
import math

PS_E = {'E1': 0.23,
        'E2': 0.23,
        'E3': 0.21,
        'E4': 0.19}

# 元素按ps降序排列
ElEMENT_LIST = ['E4', 'E3', 'E2', 'E1']

ALL_DATA_PS = {'E1': 0.23,
               'E2': 0.23,
               'E3': 0.21,
               'E4': 0.19,
               'E1#E2': 0.35,
               'E1#E3': 0.32,
               'E1#E4': 0.36,
               'E2#E3': 0.33,
               'E2#E4': 0.31,
               'E3#E4': 0.32,
               'E1#E2#E3': 0.45,
               'E1#E2#E4': 0.44,
               'E1#E3#E4': 0.42,
               'E2#E3#E4': 0.40,
               'E1#E2#E3#E4': 0.45}

visited_dict = {}

class Node():

    def __init__(self):
        # 节点名称用集合来存储
        # 通过节点名称集合的大小确定层号
        self.__node_name = [] # list

        self.__parent = None #  class node
        self.__children = [] # class node list

        # 节点的访问次数
        self.__visit_times = 0 # int
        # 收益值
        self.__Q = 0.0 # float

        # 该节点可以扩展的元素列表
        self.__expansion_element = [] # list

        self.__ps_value = 0.0 # float

    def set_node_name(self, node_name):
        if isinstance(node_name, list):
            self.__node_name = node_name
        else:
            print('the type error, list is correct')

    def get_node_name(self):
        return self.__node_name

    def set_parent(self, node):
        self.__parent = node

    def get_parent(self):
        return self.__parent

    def add_child(self, sub_node):
        self.__children.append(sub_node)

    def get_child_list(self):
        return self.__children

    def get_visit_times(self):
        return self.__visit_times

    def visit_times_add_one(self):
        self.__visit_times += 1

    def update_Q_value(self, value):
        if isinstance(value, float):
            self.__Q = value
        else:
            print('the type error, float is correct')

    def get_Q_value(self):
        return self.__Q

    def set_ps_value(self):
        # 根据节点名称获取ps值
        Q = 0
        for i in ALL_DATA_PS.keys():
            temp = copy.deepcopy(i)
            i = i.split("#")
            if set(i) == set(self.__node_name):
                Q = ALL_DATA_PS[temp]
                break

        # 用该节点的ps初始化该节点的Q
        self.__Q = float(Q)

        # 保存该节点的ps值
        self.__ps_value = float(Q)

    def get_ps_value(self):
        return self.__ps_value

    def set_expansion_element(self):

        # 判断剩余可扩展元素逻辑
        # ELEMENT_LIST 是元素按ps值升序排列的list
        temp_list = copy.deepcopy(ElEMENT_LIST)
        for i in self.__node_name:
            temp_list.remove(i)
        self.__expansion_element = temp_list

    def get_expansion_element(self):
        return self.__expansion_element

    # 判断节点是否完全扩展完毕
    def is_all_expand(self):
        if self.__expansion_element == []:
            return True
        else:
            return False

def selection(node):

    # 实现selection

    # 如果节点没有可以扩展的元素，则为叶子节点
    # 叶子节点和完全扩展节点的扩展元素列表list为空列表，
    # 最终叶子节点包含所有元素

    # 如果node是完全扩展节点，则通过UCB值确定下一个搜索的节点
    while node.is_all_expand():
        node = best_child(node)

    # 返回可扩展节点
    return node

def best_child(node):

    max_a = -1.0
    best_sub_node = None

    for sub_node in node.get_child_list():
        # 广度搜索权重
        C = math.sqrt(2.0)

        Q = sub_node.get_Q_value()
        exploration = math.log(node.get_visit_times()) / sub_node.get_visit_times()
        a = Q + C * math.sqrt(exploration)
        if a > max_a:
             max_a = a
             best_sub_node = sub_node

    return best_sub_node

def expansion(node):
    """
    实现 1判断扩展的节点是否 在本层，已经访问过，
    没访问过，创建新节点（节点名称(set)初始化，节点层号，节点父节点，节点可扩展元素List）
    访问过，将元素从扩展list中移除，按序选择下一个element
    2 如果所有element都被移除，则直接返回此节点，不再扩展，此节点变成叶子节点
    """
    # 通过node节点可扩展元素，找到本层没有访问过的节点进行扩展
    # 如果当node节点可扩展元素list为空list时，表示此node节点已经变成Leaf节点，作为叶子节点node直接返回
    while node.get_expansion_element() != []:

        # 扩展元素  expansion_element 按 元素的ps值升序
        # pop()出的元素保证是剩余元素中ps最大的
        expand_element = node.get_expansion_element().pop()

        # 扩展时，node是根节点 则node_name = []空列表
        father_node_name = copy.deepcopy(node.get_node_name())

        father_node_name.append(expand_element)

        new_node_name = father_node_name

        # 通过新节点名称，判断新节点是否已经被访问过，
        if not visited_table(new_node_name):

            new_node = Node() # 创建新节点

            new_node.set_node_name(new_node_name) # 初始化节点名称; 确定层号

            # 确定该节点可以扩展的元素
            new_node.set_expansion_element()

            new_node.set_parent(node) # new_node 添加父节点

            node.add_child(new_node) # node 添加子节点
            # 返回新扩展的基于node的子节点

            return new_node
    # node节点可扩展元素list经过判断，所有元素都被释放，list为空，退化成叶子节点返回
    return node


# # 判断节点是否在本层被防问过
def visited_table(node_name):
    """
    添加已访问的节点到visited_dict中
    判断节点node 的 节点名称，是否已经在本层的访问列表中
    """
    # 节点层号
    node_layer = len(node_name)
    node_name_set = set(node_name)
    # 确认层号
    if node_layer in visited_dict.keys():
        # 确认节点是否已经访问过了
        if node_name_set in visited_dict[node_layer]:
            return True
        else:
            visited_dict[node_layer].append(node_name_set)
            return False
    else:
        visited_dict[node_layer] = [node_name_set]
        return False

def evaluation(node):
    """
    对节点进行评估
    1.通过节点名称获取ps初始化节点Q值
    2.初始化节点访问次数
    """
    # 实现evaluation阶段

    # 获取节点集合的ps值
    # 获取节点收益值Q
    node.set_ps_value()

def backup(node):

    # 使用当前扩展节点初始化ps最佳节点
    max_ps = -1.0
    best_node = None
    # 根节点的parent node是 None
    while 1:

        # 更新节点 visit_time
        node.visit_times_add_one()

        # 退出条件：此节点是根节点,退出循环输出最佳节点，具备最高ps值节点
        if node.get_parent() == None:

            return best_node

        # 更新节点 Q
        # 获取当前节点 Q
        node_Q = node.get_Q_value()

        # 获取父节点Q
        father_node_Q = node.get_parent().get_Q_value()

        # 如果子节点收益Q大于父节收益Q则更新父节点收益Q
        # node.get_parent().get_parent() != None判断父节点不是root节点, 不对父节点的Q值进行更新
        # 更新Q值是更新search时的行动选择值
        if node_Q > father_node_Q and node.get_parent().get_parent() != None:
            # 更新父节点收益Q
            node.get_parent().update_Q_value(node_Q)

            # 更新本条路径上的ps最大的节点
        if node.get_ps_value() > max_ps:
            # 更新ps最值
            max_ps = node.get_ps_value()

            # 更新最佳节点
            best_node = node

        # 更新当前处理节点
        node = node.get_parent()

def monte_carlo_tree_search(node):

    # 判断整颗MCT树是否完全搜索完毕
    tree_search_terminal = False

    # 选择 包含了使用UCB值搜索完全扩展节点过程
    expansion_node = selection(node)
    # 扩展
    evaluation_node = expansion(expansion_node)
    # 模拟
    evaluation(evaluation_node)
    # 回溯
    # 确定本条路径上ps值最大的node
    best_node = backup(evaluation_node)

    # 如果evaluation_node是最终的叶子节点，
    # 则node_name包含的元素与搜索对象cuboid元素相同
    # 整颗MCT全部搜索完毕
    if len(evaluation_node.get_node_name()) == len(ElEMENT_LIST):
        final_leaf_node = expansion_node
        tree_search_terminal = True
        return [final_leaf_node, tree_search_terminal]

    return [best_node, tree_search_terminal]

def main():

    # 初始化根节点
    root_node = Node()

    # 初始化根节点可扩展元素
    root_node.set_expansion_element()

    # 最大迭代次数
    M = 100

    # 阈值
    PT = 0.50

    real_M = 0
    total_best_node = root_node
    for i in range(M):
        # 每次迭代蒙特卡罗搜索都会在搜索路径上找到到收益Q最大的局部最优解
        # 每次搜索从根节点出发
        info_list = monte_carlo_tree_search(root_node)

        # 获取局部最佳节点
        local_best_node = info_list[0]

        # 获取MTC完全搜索标志位
        tree_search_terminal = info_list[1]

        # 实际运行次数
        real_M += 1

        # 如果局部最优解ps大于全局最优解ps，则更新全局最优解ps
        if local_best_node.get_ps_value() > total_best_node.get_ps_value():
            total_best_node = local_best_node

        # 大于域值PT返回
        if total_best_node.get_ps_value() >= PT:
            print('real_M:', real_M)
            return total_best_node.get_node_name()

        # MCT完全搜索结束
        if tree_search_terminal:
            print('real_M:', real_M)
            return total_best_node.get_node_name()

    # 大于迭代时间M返回
    print('real_M:', real_M)
    return total_best_node.get_node_name()

if __name__ == '__main__':

    # tree = treemap.TreeMap(1536792600000)
    # tree.createMap()
    # ElEMENT_LIST = tree.cuboidInstanceList(['c'])

    x = main()
    print(x)
    print(visited_dict)

real_M: 17
['E3', 'E1', 'E2']
{1: [{'E1'}, {'E2'}, {'E3'}, {'E4'}], 2: [{'E1', 'E2'}, {'E3', 'E2'}, {'E3', 'E1'}, {'E1', 'E4'}, {'E4', 'E2'}, {'E3', 'E4'}], 3: [{'E3', 'E2', 'E1'}, {'E3', 'E4', 'E1'}, {'E1', 'E2', 'E4'}], 4: [{'E1', 'E2', 'E4', 'E3'}]}
